# Hente hemmeligheter fra secret manager

In [5]:
from google.cloud import secretmanager

In [6]:
secrets = secretmanager.SecretManagerServiceClient()

In [7]:
project_name = "knada-gcp"
secret_name = "vebjorn-rekkebo-bac1"
version = "latest"

resource_name = f"projects/{project_name}/secrets/{secret_name}/versions/{version}"
secret = secrets.access_secret_version(name=resource_name)

In [8]:
secrets = dict([line.split(":") for line in secret.payload.data.decode('UTF-8').splitlines()])

In [10]:
import psycopg2

import pandas as pd
import datetime

from sqlalchemy import create_engine

In [11]:
method = "postgresql+psycopg2"
host = "A01DBVL029.adeo.no:5432"
db_name = "pensjon-psak-db"

alchemyEngine = create_engine(f'{method}://{secrets["PSAK_POSTGRES_USER"]}:{secrets["PSAK_POSTGRES_PASSWORD"]}@{host}/{db_name}')
dbConnection = alchemyEngine.connect()

In [12]:
df_dbstats = pd.read_sql(
    """
    select column_name, data_type, character_maximum_length, column_default, is_nullable
    from INFORMATION_SCHEMA.COLUMNS where table_name = 't_saksbeh_preferanser';
    """,
    dbConnection)

In [13]:
df_dbstats.head()

,column_name,data_type,character_maximum_length,column_default,is_nullable
0,dato_opprettet,timestamp without time zone,NaN,None,NO
1,versjon,numeric,NaN,None,NO
2,dato_endret,timestamp without time zone,NaN,None,NO
3,bruk_nytt_design,boolean,NaN,None,YES
4,endret_av,character varying,20.0,None,NO


In [14]:
df_pref = pd.read_sql(
    """
    select bruk_nytt_design, count(*) Antall
    from t_saksbeh_preferanser
    group by bruk_nytt_design;
    """,
    dbConnection).T

In [15]:
new_header = df_pref.iloc[0]
df_pref = df_pref[1:]
df_pref.columns = new_header
df_pref["Uttrekk_tidspunkt"] = datetime.datetime.utcnow().isoformat()

df_pref.columns = ["Uspesifisert", "Gammelt", "Nytt", "Uttrekk_tidspunkt"]
df_pref = df_pref.reset_index(drop=True)

In [16]:
df_pref["Andel_nytt"] = df_pref.Nytt / df_pref[["Uspesifisert", "Gammelt", "Nytt"]].sum(axis=1)
df_pref["Andel_nytt"] = df_pref["Andel_nytt"].apply(lambda x: round(x, 2))

In [17]:
df_pref.head()

,Uspesifisert,Gammelt,Nytt,Uttrekk_tidspunkt,Andel_nytt
0,140,287,236,2022-10-06T09:57:18.007797,0.36


In [19]:
from google.cloud.bigquery import Client, LoadJobConfig

table_id = f'pensjon-saksbehandli-prod-1f83.saksbehandling_psak.nytt_gammelt_design'
job_config = LoadJobConfig(
    write_disposition="WRITE_APPEND",
)

client = Client(project="pensjon-saksbehandli-prod-1f83")

job = client.load_table_from_dataframe(df_pref, table_id, job_config=job_config)
job.result()

LoadJob<project=pensjon-saksbehandli-prod-1f83, location=europe-north1, id=93ab617e-770d-461b-8581-ca119f2aa320>